In [25]:
import pandas as pd
import numpy as np
from keras import Sequential
from keras.src.callbacks import LearningRateScheduler
from keras.src.layers import Dense
from keras.src.optimizers import SGD
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go

In [26]:
data = pd.read_csv("data.csv").drop("Unnamed: 0", axis=1)
labels = data["value"]
data = data["year"]
temp = data

labels = np.log2(labels)

mean, std = data.mean(), data.std()
data = data - data.mean()
data = data / data.std()

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2)

In [27]:
model = Sequential()
model.add(Dense(units=1, input_shape=(1,)))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1)                 2         
                                                                 
Total params: 2 (8.00 Byte)
Trainable params: 2 (8.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
model.compile(optimizer=SGD(0.001, 0.9),
              loss='mse')


def schedule(epoch, lr):
    if epoch < 25:
        return 0.001
    else:
        return 0.0001


lrs = LearningRateScheduler(schedule)

history = model.fit(train_data, train_labels, epochs=50, callbacks=[lrs])

Epoch 1/50
9/9 [==============================] - 1s 4ms/step - loss: 83.0854 - lr: 0.0010
Epoch 2/50
9/9 [==============================] - 0s 5ms/step - loss: 67.3596 - lr: 0.0010
Epoch 3/50
9/9 [==============================] - 0s 5ms/step - loss: 49.0119 - lr: 0.0010
Epoch 4/50
9/9 [==============================] - 0s 5ms/step - loss: 33.5428 - lr: 0.0010
Epoch 5/50
9/9 [==============================] - 0s 5ms/step - loss: 22.4462 - lr: 0.0010
Epoch 6/50
9/9 [==============================] - 0s 5ms/step - loss: 14.7776 - lr: 0.0010
Epoch 7/50
9/9 [==============================] - 0s 9ms/step - loss: 9.5940 - lr: 0.0010
Epoch 8/50
9/9 [==============================] - 0s 6ms/step - loss: 6.3630 - lr: 0.0010
Epoch 9/50
9/9 [==============================] - 0s 6ms/step - loss: 4.3217 - lr: 0.0010
Epoch 10/50
9/9 [==============================] - 0s 4ms/step - loss: 3.0432 - lr: 0.0010
Epoch 11/50
9/9 [==============================] - 0s 3ms/step - loss: 2.2167 - lr: 0.0010
Ep

In [29]:
def draw_history(history):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history["loss"], name="mse"))
    # fig.add_trace(go.Scatter(x=history.epoch, y=history.history["val_loss"], name="val_mse"))
    fig.show()


draw_history(history)

In [30]:
model.evaluate(test_data, test_labels)

3/3 [==============================] - 0s 4ms/step - loss: 0.7725


0.7725247144699097

In [31]:
weights = model.layers[0].get_weights()

x = np.linspace(-2, 1.5, 300)
y = weights[0][0, 0] * x + weights[1][0]

print("y = ax + b")
print(f"a = {weights[0][0, 0]}")
print(f"b = {weights[1][0]}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=data, y=labels, name="data", mode="markers"))
fig.add_trace(go.Scatter(x=x, y=y, name="approximation", mode="lines"))
fig.show()

go.Figure(
    [go.Scatter(x=temp, y=2 ** labels, mode="markers"),
     go.Scatter(x=(x * std) + mean, y=2 ** y, name="approximation", mode="lines")]
).show()

y = ax + b
a = 3.1568052768707275
b = 8.320748329162598


In [33]:
data_to_predict = [1900, 1950, 2000, 2020]

standarized_data = (data_to_predict - mean) / std

preds = model.predict(standarized_data, verbose=0).ravel()

df = pd.DataFrame({
    "year_to_predict": data_to_predict,
    "stded_year": standarized_data,
    "pred": preds,
    "exponential": 2 ** preds
})

df

,year_to_predict,stded_year,pred,exponential
0,1900,-1.980318,2.069270,4.196742
1,1950,-0.592321,6.450907,87.481575
2,2000,0.795677,10.832544,1823.562988
3,2020,1.350876,12.585199,6145.008789
